
# Ayudantía 09: Threading
#### Autores: Nicolás Quiroz (`@naquiroz`) y Daniel Pinto (`@drpinto1`)

[Link](https://docs.google.com/forms/d/e/1FAIpQLSdNDAXi5-LDcDytG18WN57PbHT7MrnNZZJ9ZflfCoQ0Uv_aAw/viewform) al feedback de la ayudantía :D

**¿Qué es un thread?**

Una secuencia muy puequeña, de tareas encadenadas, que puede ser ejecutada por un sistema operativo, eso significa, básicamente, que un thread es solamente, una sección de código.

¿En qué se diferencia eso de lo que hemos estado haciendo hasta ahora?

Hasta ahora, sin saberlo, han usado threads.. o para ser precisos han usado _**un thread**_ y lo que aprenderán esta semana, es a manejar múltiples threads.


Esto es una herramienta muy poderosa, que les permitirá crear diferentes tipos de programas, por ejemplo, juegos!

Imaginemos que queremos modelar a un enemigo en un juego, que cambia su posición cada 1 segundo.
Esto lo podemos hacer más o menos así:

In [ ]:
import time
import random

class Enemigo:
    def __init__(self, nombre, x, y):
        self.x = x
        self.y = y
        self.nombre = nombre
        
    def mover(self):
        self.x += random.randint(0, 5)
        self.y += random.randint(0, 5)
        print(f'{self.nombre}: Me moví a {self.x}, {self.y}')
        

el_malo = Enemigo('el_malo', 0, 0)

for _ in range(5):
    el_malo.mover()
    print()
    time.sleep(1)

¿Qué hacemos si queremos varios enemigos?

Podemos hacer, básicamente, lo mismo

In [ ]:
el_malo = Enemigo('el_malo', 0, 0)
el_muy_malo = Enemigo('el_muy_malo', 0, 0)
el_mas_malo = Enemigo('el_mas_malo', 0, 0)

for _ in range(3):
    el_malo.mover()
    el_muy_malo.mover()
    el_mas_malo.mover()
    print()
    time.sleep(1)

¿Qué pasa si queremos agregar un jugador?

Queremos que el jugador también se pueda mover cada un segundo, así que seguimos la misma lógica

In [ ]:
class Jugador:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def mover(self):
        mov = int(input('Seleccione un número de casillas para mover. '))
        self.x += mov
        self.y += mov
        print(f'jugador: Me moví a {self.x}, {self.y}')

jugador = Jugador(0, 0)
el_malo = Enemigo('el_malo', 0, 0)
el_muy_malo = Enemigo('el_muy_malo', 0, 0)
el_mas_malo = Enemigo('el_mas_malo', 0, 0)

for _ in range(3):
    jugador.mover()
    el_malo.mover()
    el_muy_malo.mover()
    el_mas_malo.mover()
    print()
    time.sleep(1)

¿Cuál es el problema con esto?
Los enemigos solo se mueven luego de haber movido al jugador, no cada un segundo

¿Cómo podemos arreglar esto?
Por supuesto que con la ayuda de threads.

Comenzamos con la sintáxis básica para threads

In [ ]:
import threading as thr

def thredeador():
    wait= random.randint(1, 4)
    mi_thread = thr.current_thread()
    print(f'Hola, soy el thread {mi_thread.name}, esperare {wait}s')
    time.sleep(wait)
    print(f'Thread {mi_thread.name} terminando x__x')

uno = thr.Thread(name='thread uno', target=thredeador)
dos = thr.Thread(name='thread dos', target=thredeador)

uno.start()
dos.start()

Ahora, volvemos a nuestro juego

#### IMPORTANTE:
Los siguientes bloques de código pueden caerse luego de ingresar el primer input, sin embargo si copias el mismo código a un archivo .py y lo ejecutas normalmente va a funcionar como se espera; esto se debe a la manera en que jupyter maneja los threads.

In [ ]:
def movedor(movible):  # duck typing ;)
    for _ in range(3):
        movible.mover()
        time.sleep(1)

el_malo = Enemigo('el_malo', 0, 0)
el_muy_malo = Enemigo('el_muy_malo', 0, 0)
el_malisimo = Enemigo('el_malisimo', 0, 0)
jugador = Jugador(0, 0)

mueve0 = thr.Thread(target=movedor(jugador))  # Podemos omitir el nombre del thread
mueve1 = thr.Thread(target=movedor(el_malo))
mueve2 = thr.Thread(target=movedor(el_muy_malo))
mueve3 = thr.Thread(target=movedor(el_malisimo))

*¿Qué está pasando?* **😩**

Solamente tenemos un ~~no tan~~ pequeño error de sintaxis al momento de declarar el target. La sintaxis correcta es la siguiente:

In [ ]:
mueve0 = thr.Thread(target=movedor, args=(jugador,))
mueve1 = thr.Thread(target=movedor, args=(el_malo,))
mueve2 = thr.Thread(target=movedor, args=(el_muy_malo,))
mueve3 = thr.Thread(target=movedor, args=(el_mas_malo,))

mueve_todo = [mueve0, mueve1, mueve2, mueve3]
for mueve in mueve_todo:
    mueve.start()

Notamos que para llamar a una función con argumentos desde el Thread, necesitamos darle la función target, __sin argumentos ni paréntesis__ y una __tupla__ con los argumentos, por lo que si es solo uno, debemos crear una tupla de largo 1 con el argumento y una coma

Podemos hacer lo mismo, aplicando materia anterior, en conjunto con threading. Con la ayuda de nuestra gran amiga **Herencia**

Si se dieron cuenta en los códigos anteriores, utilizamos thr.Thread, en lugar de thr.thread

Esto es una sutileza, sin embargo, si miramos el código de estilos _PEP8_ ~~yay!~~, encontraremos que lo anterior representa una clase, o sea, _podemos heredar_ de Thread 😮

In [ ]:
class Enemigo(thr.Thread):
    def __init__(self, nombre, x, y, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.x = x
        self.y = y
        self.nombre = nombre
        
    def mover(self):  # mismo mover :o
        self.x += random.randint(0, 5)
        self.y += random.randint(0, 5)
        print(f'{self.nombre}: Me moví a {self.x}, {self.y}')
        
    def run(self):
        for i in range(3):
            self.mover()
            time.sleep(random.randint(0, 3))

class Jugador(thr.Thread):
    def __init__(self, x, y, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.x = x
        self.y = y

    def mover(self):  # Mismo mover que antes
        mov = int(input('Seleccione un número de casillas para mover.'))
        self.x += mov
        self.y += mov
        print(f'jugador: Me moví a {self.x}, {self.y}')
        
    def run(self):
        for i in range(3):
            self.mover()
            time.sleep(random.randint(0, 3))

el_malo = Enemigo('el_malo', 0, 0)
el_muy_malo = Enemigo('el_muy_malo', 0, 0)
el_malisimo = Enemigo('el_malisimo', 0, 0)
jugador = Jugador(5, 2)

el_malo.start()
el_muy_malo.start()
el_malisimo.start()
jugador.start()

Agreguemos ahora una pequeña funcionalidad, el __*Game Over*__

¿Qué pasa si queremos que el programa termine luego de que el jugador se mueva solamente una vez?

Podemos redefinir el run de nuestro jugador, para que funcione una única vez en lugar de 3.

In [ ]:
class Jugador(thr.Thread):
    def __init__(self, x, y, *args, **kwargs):  # mismo init
        super().__init__(*args, **kwargs)
        self.x = x
        self.y = y

    def mover(self):  # Mismo mover
        mov = int(input('Seleccione un número de casillas para mover.'))
        self.x += mov
        self.y += mov
        print(f'jugador: Me moví a {self.x}, {self.y}')
        
    def run(self):  # quitamos el for del run
        self.mover()
        
el_malo = Enemigo('el_malo', 0, 0)
el_muy_malo = Enemigo('el_muy_malo', 0, 0)
el_malisimo = Enemigo('el_malisimo', 0, 0)
jugador = Jugador(5, 2)

el_malo.start()
el_muy_malo.start()
el_malisimo.start()
jugador.start()

Notamos que el thread termina, pero el programa principal continua su ejecución.

Esto se puede arreglar utilizando *daemon_threads*, que son exáctamente lo que queremos, threads que no impiden que el programa principal termine su ejecución si es que estos no han terminado; los podemos implementar de dos maneras:

#### IMPORTANTE:
Los siguientes dos bloques de código no van a funcionar, sin embargo si copias el mismo código a un archivo .py y lo ejecutas normalmente va a funcionar como se espera; esto se debe a la manera en que jupyter maneja los threads. Ocurre lo mismo con el ejercicio final.

### Manera 1:
Sin alterar la definición de las clases

In [ ]:
# Misma instanciación
el_malo = Enemigo('el_malo', 0, 0)
el_muy_malo = Enemigo('el_muy_malo', 0, 0)
el_malisimo = Enemigo('el_malisimo', 0, 0)
jugador = Jugador(5, 2)

# Agregamos las lineas convirtiendo los threads
el_malo.daemon = True
el_muy_malo.daemon = True
el_malisimo.daemon = True

# Iniciamos
el_malo.start()
el_muy_malo.start()
el_malisimo.start()
jugador.start()

### Manera 2:
Declarándolo en la definición de la clase.

In [ ]:
# Esta definción es exactamente la misma
class Enemigo(thr.Thread):
    def __init__(self, nombre, x, y, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.x = x
        self.y = y
        self.nombre = nombre
        self.daemon = True  # Excepto por esta linea, donde decalramos al thread como Daemon
        
    def mover(self):
        self.x += random.randint(0, 5)
        self.y += random.randint(0, 5)
        print(f'{self.nombre}: Me moví a {self.x}, {self.y}')
        
    def run(self):
        for i in range(3):
            self.mover()
            time.sleep(random.randint(0, 3))
            
# Instanciamos
el_malo = Enemigo('el_malo', 0, 0)
el_muy_malo = Enemigo('el_muy_malo', 0, 0)
el_malisimo = Enemigo('el_malisimo', 0, 0)
jugador = Jugador(5, 2)

# Iniciamos
el_malo.start()
el_muy_malo.start()
el_malisimo.start()
jugador.start()

Otra herramienta poderosa, que hace algo "opuesto" a _daemon_ es __join()__; el propósito de esta función es detener la ejecución del programa principal hasta que termine de ejecutarse el thread que llamo a join; este comportamiento se puede observar a continuación.

In [ ]:
def esperemos_un_rato():
    """ La funcion espera 5 segundos, haciendo un print por segundo """
    timer = 0
    for i in range(5):
        print(f'Esperando {timer}')
        timer += 1
        time.sleep(1)

esperador = thr.Thread(target = esperemos_un_rato)  # creamos el thread
esperador.start()  # Iniciamos el thread

for i in range(5):
    if i == 2:  # detenemos la ejecución del for
        esperador.join()  # hasta que termine el thread "esperador"
    print(f'Thread Principal: {i}')

La última herramienta de threads que queremos mencionar, y la que te permite sincronizar la ejecución de varios threads, es threading.Lock y su aplicación más útil es __manejar variables/recursos compartidos por varios threads__.
En la práctica, lo que hace locks es prohibir el acceso a cierto recurso a todos los threads, _excepto a uno_ a la vez, de esta manera, si uno de tus threads está haciendo uso de una variable/archivo/etc los demás no podrán realizar cambios hasta que el thread actual termine de hacerlo.
Algunos ejmplos del funcionamiento de locks se encuentran más adelante.

Con todo lo anterior, podemos ir aún más alla con threads y simular algo más realista, algo más cercano a ustedes y lo que viven todos los días. Algo como...

Una pelea de robots!

## AC21- 2030-3 : PDR

Los robots han invadido al DCC!. Debido a causas totalmente creíbles y con mucho sentido, te ves obligado a simular una pelea de robots con el objetivo de defender el departamento!.
Sin embargo, porque ~~no nos sirve tanto para explicar la materia~~ el dcc no quiere fomentar la clandestinidad, la pelea no será simplemente una lucha entre robots iguales, si no una alianza entre varios equipos de robots pequeños, para vencer un robot gigante que está intentando destruir el mundo (nuevamente, causa totalmente creíble y con sentido). 

Deberás modelar esta batalla entre 5 robots (cada uno con un estratega y piloto correspondiente) en un mapa de 100 x 100 coordenadas (las coordenadas van de 0 a 100 en cada eje). Por lo tanto, los robots deben poseer coordenadas de su posición actual.

Condiciones iniciales.

- Los robots comienzan en posiciones aleatorias
- El mega robot comienza en posiciones aleatorias.

#### El Mega Robot

El mega robot es el robot gigante con el cual deberán pelear. Este comienza con 5000 de vida, y solo ataca cuando es atacado, atacando siempre 100 de vida. Por otro lado, cada vez que este robot pierde 500 o más de vida, debe moverse a una nueva ubicación aleatoria en el mapa.

El mega robot posee además una función `vulnerate(self, hash_num: int)` cuya existencia se justifica ~~para demostrar los contenidos mas adelante~~ con un accidente del diseñador del robot. En esta se ingresa una clave de hash (un número simplemente, que si es igual a la clave secreta del `MegaRobot`, este pierte 1000 de vida instantáneamente)

In [ ]:
import threading as thr
import random
import time

class MegaRobot:
    def __init__(self):
        self.pos_lock = thr.Lock()
        self.life_lock = thr.Lock()
        self._life = 5 * 10**3
        self.lf_check_pt = self._life  # última vez que perdió >= 500, se usa para checquear cuando debe moverse
        self._x = random.randint(0, 99)
        self._y = random.randint(0, 99)
        self._secret_hash = random.randint(0, 50)
    
    def vulnerate(self, hash_num: int):
        """ Revisa si la clave es correcta y hace el daño correspondiente"""
        if self._secret_hash == hash_num:
            print('Uh oh, MegaRobot vulnerado!')
            self.life -= 1000
    
    def move(self, x: int, y: int):
        self.x = x
        self.y = y

    def move_rand(self):
        self.move(random.randint(0, 99), random.randint(0, 99))
    
    # Notar el uso de locks en todas estas properties
    @property
    def x(self):
        with self.pos_lock:
            return self._x

    @property
    def y(self):
        with self.pos_lock:
            return self._y

    @x.setter
    def x(self, value):
        with self.pos_lock:
            self._x = max(0, min(99, value))

    @y.setter
    def y(self, value):
        with self.pos_lock:
            self._y = max(0, min(99, value))

    @property
    def life(self):
        with self.life_lock:
            return self._life

    @life.setter
    def life(self, value: int):
        with self.life_lock:
            new_life = min(max(0, value), 5000)
            if self.lf_check_pt - new_life >= 500:
                self.lf_check_pt = new_life
                self.move_rand()
                print(f'El Mega Robot ha cambiado su ubicación! Ahora está en ({self.x},{self.y})')
            else:
                print(f'Mega robot atacado en ({self.x}, {self.y})')
            self._life = new_life

#### Los Robots
Cada robot pequeño será manejado por un equipo de dos personas, un piloto y un estratega. Tienen 2000 de vida inicial y cada vez que atacan, infligen una cantidad aleatoria de daño, con valores entre 0 y 250.

Como se esperaría, el piloto es el encargado de hacer que el robot se desplace y ataque, mientras que el estratega se encargará de buscar el punto débil del robot enemigo, para así ganar la pelea.

In [ ]:
from itertools import count
class Robot:
    id_gen = count()  # Generamos ID unico para cada robot
    def __init__(self, x: int, y: int):
        self.pos_lock = thr.Lock()  # lock para posicion
        self.life_lock = thr.Lock()  # lock para vida
        self.robot_id = next(self.id_gen)
        self._x = x
        self._y = y
        self._life = 2*10**3

    def attempt_vulneration(self, mega_robot: MegaRobot, value: int):
        mega_robot.vulnerate(value)
        
    def displace(self, x: int, y: int):
        '''Desplaza al robot. No lo mueve a (x,y), lo desplaza un delta x y un delta y'''
        with self.pos_lock:
            self.x += x
            self.y += y
        
    def attack(self, mega_robot: MegaRobot):
        print(f'Robot {self.robot_id} atacando a Mega Robot')
        mega_robot.life -= random.randint(0, 250)

    @property
    def life(self):
        with self.life_lock:
            return self._life

    @life.setter
    def life(self, value):
        with self.life_lock:
            self._life = min(max(0, value), 2000)

    def __repr__(self):
        return f'Robot {self.robot_id}'

#### El Piloto
El piloto será el encargado de cambiar la posición del robot cada cierto tiempo, entre 1 y 5 segundos, además de enviar la orden al robot, de intentar atacar, cada 2 movimientos de este. También, debido a la baja tecnología de nuestro planeta, los robots pueden moverse un bloque a la vez, en dirección aleatoria. Cada vez que el piloto decide enviar a su robot a atacar, deberá emitir una notificación al estratega, que es momento de intentar vulnerar al MegaRobot.

Es importante mencionar que el piloto solo puede atacar cuando la distancia euclidiana entre su robot y el MegaRobot es menor a 70.

In [ ]:
from itertools import cycle  # cycle crea un generador infinito que repite un iterable

class Piloto(thr.Thread):
    def __init__(self, robot: Robot, mega_robot: MegaRobot, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.robot = robot
        self.mega = mega_robot
        self.daemon = True
        self.fired_signal = thr.Event()  # señal para avisar al estratega
    
    @property
    def euclidean(self):
        x = self.robot.x - self.mega.x
        y = self.robot.y - self.mega.y
        return (x**2 + y ** 2) ** 0.5
    
    def run(self):
        moves = cycle(range(1,3))  # gen(1, 2, 1, 2, 1, 2, ...)
        for move in moves:
            if not self.robot.life or not self.mega.life:
                return  # Si alguno muere, el run retorna y el thread termina

            time.sleep(random.randint(1, 5))
            dirs = ((0,1), (1,0), (0, -1), (-1, 0))
            self.robot.displace(*random.choice(dirs))  # Se mueve
            if move == 2 and self.euclidean < 70:  # Intenta atacar/vulnerar si es posible
                self.robot.attack(self.mega)
                self.robot.life -= 400
                self.fired_signal.set()  # avisa al estratega

#### El Estratega
El estratega deberá observar cómo se desarrolla la pelea buscando puntos débiles del mega robot. Cuando el robot asociado a un estratega, ataca, este intentará vulnerar al MegRobot. Para esto debe usar la función provista por el MegaRobot `vulnerate` con un número aleatorio para intentar vulnerar las defensas del MegaRobot.

Nuevamente, **solo es posible vulnerar al MegaRobot cuando el robot asociado al estratega, ataca.**. La única forma que el estratega se de cuenta de esto, es a través del Piloto, quien le avisará cuando puede vulnerar.

In [ ]:
class Estratega(thr.Thread):
    def __init__(self, robot: Robot, mega_robot: MegaRobot, signal: thr.Event, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.robot = robot
        self.mega_robot = mega_robot
        self.daemon = True
        self.fired_signal = signal  # misma señal que el piloto del mismo robot

    def run(self):
        while self.robot.life and self.mega_robot.life:  # continua mientras viva y viva el enemigo
            self.fired_signal.wait()  # espera la señal del piloto
            print('Intentando vulnerar mega robot...')
            self.robot.attempt_vulneration(self.mega_robot, random.randint(0, 100))   
            self.fired_signal.clear()  # reinicia la señal para que se pueda seguir usando

Finalmente, cuando el Mega Robot, haya perdido 100% de su vida, debes terminar la batalla y retornar todos los robots vencedores.

In [ ]:
def sim_battle():
    mega = MegaRobot()
    estrategas = []
    pilots = []
    robots = []
    
    # Crea las instancias de Robots, Pilotos y Estrategas
    for _ in range(5):
        robot  = Robot(random.randint(0,99), random.randint(0, 99))
        robots.append(robot)
        pilot = Piloto(robot, mega)
        pilots.append(pilot)
        estrategas.append(Estratega(robot, mega, pilot.fired_signal))
    
    # Inicia los threads para estrategas y pilotos
    for e, p in zip(estrategas, pilots):
        p.start()
        e.start()
    
    # Dado que todos nuestros threads son daemon es necesario hacer join para que el programa funcione
    for e, p in zip(estrategas, pilots):
        e.join()
        p.join()
        
    # Cuando todos los threads anteriores Pilotos/Estrategas terminen, es porque se acabó la batalla y se puede
    # continuar ejecutando, recordar que solo se llega a esta linea si todos terminan por los join anteriores
    print('Batalla terminada!')
    return list(filter(lambda r: bool(r.life), robots))

In [ ]:
sim_battle()